In [3]:
import pandas as pd
import math
from schedule import every, repeat, run_pending
import time
import time
from datetime import datetime, timedelta
from sms import send_sms

def convert_to_yolo_format(x1, y1, x2, y2, image_width, image_height):
    # Calculate center coordinates
    x = (x1 + x2) / 2
    y = (y1 + y2) / 2
    
    # Calculate width and height
    w = x2 - x1
    h = y2 - y1
    
    # Normalize coordinates and dimensions
    x /= image_width
    y /= image_height
    w /= image_width
    h /= image_height
    
    return x, y, w, h
print("doing job")
    # Read the CSV file into a DataFrame
df = pd.read_csv('goldfish_positions2.csv')
    # Convert bounding box coordinates to YOLO style

df_yolo = df.apply(lambda row: pd.Series(convert_to_yolo_format(row['x1'], row['y1'], row['x2'], row['y2'], 1280, 960)), axis=1)


# Rename the columns for clarity
df_yolo.columns = ['x', 'y', 'w', 'h']

# Concatenate the new DataFrame with the original DataFrame
df = pd.concat([df, df_yolo], axis=1)
df




        





doing job


,timestamp,x1,y1,x2,y2,score,class,x,y,w,h
0,2024-02-01 16:44:19.553899,94.933685,132.086365,194.737762,223.593658,0.789632,goldfish,0.113153,0.185250,0.077972,0.095320
1,2024-02-01 16:44:20.657760,95.265366,130.684601,196.366318,225.822479,0.832798,goldfish,0.113919,0.185681,0.078985,0.099102
2,2024-02-01 16:44:21.778035,96.827110,126.959656,195.030670,228.109375,0.796487,goldfish,0.114007,0.184932,0.076722,0.105364
3,2024-02-01 16:44:23.170660,588.770630,10.304977,640.000000,94.865738,0.546307,goldfish,0.479989,0.054776,0.040023,0.088084
4,2024-02-01 16:44:24.258731,144.089874,93.359245,265.570221,196.924194,0.626299,goldfish,0.160023,0.151189,0.094907,0.107880
...,...,...,...,...,...,...,...,...,...,...,...
1885,2024-02-01 17:35:14.930497,112.527245,175.033112,202.444000,258.053741,0.911787,goldfish,0.123036,0.225566,0.070247,0.086480
1886,2024-02-01 17:35:16.088682,113.303993,176.047516,202.652130,258.590088,0.930018,goldfish,0.123420,0.226374,0.069803,0.085982
1887,2024-02-01 17:35:17.192355,112.353584,176.813507,200.996460,258.980927,0.911322,goldfish,0.122402,0.226976,0.069252,0.085591
1888,2024-02-01 17:35:18.300208,112.445862,176.674469,201.738434,259.403748,0.927356,goldfish,0.122728,0.227124,0.069760,0.086176


In [4]:
# Convert 'timestamp' column to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Calculate the distance between consecutive points


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1890 entries, 0 to 1889
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  1890 non-null   datetime64[ns]
 1   x1         1890 non-null   float64       
 2   y1         1890 non-null   float64       
 3   x2         1890 non-null   float64       
 4   y2         1890 non-null   float64       
 5   score      1890 non-null   float64       
 6   class      1890 non-null   object        
 7   x          1890 non-null   float64       
 8   y          1890 non-null   float64       
 9   w          1890 non-null   float64       
 10  h          1890 non-null   float64       
dtypes: datetime64[ns](1), float64(9), object(1)
memory usage: 162.6+ KB


In [10]:
df['x']

0       0.113153
1       0.113919
2       0.114007
3       0.479989
4       0.160023
          ...   
1885    0.123036
1886    0.123420
1887    0.122402
1888    0.122728
1889    0.121779
Name: x, Length: 1890, dtype: float64

In [9]:
df['x'].shift(1)

0            NaN
1       0.113153
2       0.113919
3       0.114007
4       0.479989
          ...   
1885    0.122749
1886    0.123036
1887    0.123420
1888    0.122402
1889    0.122728
Name: x, Length: 1890, dtype: float64

In [14]:
import numpy as np
df["x_lag"] = df["x"].shift(1)
df["y_lag"] = df["y"].shift(1)


distance= np.sqrt((df['x']-df["x_lag"])**2+(df['y']-df["y_lag"])**2)


In [16]:
distance

0            NaN
1       0.000879
2       0.000754
3       0.388437
4       0.334175
          ...   
1885    0.000478
1886    0.000895
1887    0.001183
1888    0.000358
1889    0.000952
Length: 1890, dtype: float64

In [5]:

mean_dist = df['distance'].mean()

# Get the current time
current_time = datetime.now()

# Filter data from the past hour
past_hour_data = df[df['timestamp'] > current_time - timedelta(hours=1)]

# Calculate the mean distance for the past hour
dist_past_hour = past_hour_data['distance'].mean()
# Compare the mean distance of the past hour with the overall mean distance

TypeError: cannot convert the series to <class 'float'>

In [ ]:
    sort_y1=df.sort_values("y1",ascending=True)["y1"]
    mean_hauteur=sort_y1.mean()
    

    bottom=mean_hauteur*0.3
    # Add a new column to the DataFrame based on the condition
    past_hour_data['is_below_bottom'] = (past_hour_data['y1'] + past_hour_data['y2']) / 2 < bottom

    
    # Display the DataFrame with the added column
    true_count = past_hour_data['is_below_bottom'].sum()
    print(dist_past_hour)
    print(true_count)
    if dist_past_hour < 10000:#mean_dist *0.3 :
        #send_sms("Your fish is in bad condition")
        sleep_mode = True
        print("distance verification")
    if true_count > 1:#len(past_hour_data)*0.5:
        #send_sms("the fish is at the bottom for a long time")
        print("bottom verification")
    if sleep_mode :
        time.sleep(100000)

In [17]:
hauteur_mean_1.iloc[2:12].mean()

67.10288925170899

In [18]:
hauteur_mean_2.iloc[2:12].mean()

126.83258361816407

In [28]:
sort_y1=df.sort_values("y1",ascending=True)["y1"]
sort_y2=df.sort_values("y2",ascending=True)["y2"]
mean_y1=sort_y1.mean()
mean_y2=sort_y2.mean()
mean_hauteur=(mean_y1+mean_y2)/2

bottom=mean_hauteur*0.6
# Add a new column to the DataFrame based on the condition
past_hour_data['is_below_bottom'] = (past_hour_data['y1'] + past_hour_data['y2']) / 2 < bottom

# Display the DataFrame with the added column
print(past_hour_data)

                      timestamp          x1          y1          x2  \
0    2024-01-31 12:51:26.329014  236.313110    2.982536  349.453186   
1    2024-01-31 12:51:28.958485  116.576157  265.301575  251.054245   
2    2024-01-31 12:51:29.981496   91.635086  245.824722  196.740250   
3    2024-01-31 12:51:31.018855   52.241882  185.532440  160.304535   
4    2024-01-31 12:51:32.028476   86.993744  164.472992  180.061340   
...                         ...         ...         ...         ...   
1152 2024-01-31 13:41:03.253670   26.676453  211.736542   76.431236   
1153 2024-01-31 13:41:04.263928   32.134171  194.452759   86.658073   
1154 2024-01-31 13:41:09.271143   51.341309  204.382538  132.903519   
1155 2024-01-31 13:41:13.640489   81.424133  195.694611  139.759338   
1156 2024-01-31 13:41:14.665051   91.248260  189.333984  163.793503   

              y2     score     class    distance  is_below_bottom  
0      44.961021  0.517179  goldfish  120.676717             True  
1     357.6

: 

In [27]:
true_count = past_hour_data['is_below_bottom'].sum()

# Display the count
print(f"Number of True values: {true_count}")

Number of True values: 748
